# Segmenting and Clustering Neighborhoods in Toronto

## Part 1: Create the dataframe [](#item1) 

Import necessary libraries

In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup
import requests

Read html file using `pandas`, set "Not assigned" as NA values, and drop the rows with missing Borough values

In [2]:
url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
df = pd.read_html(url, na_values="Not assigned")[0].dropna(subset=['Borough'])

In [3]:
df.head()

,Postal Code,Borough,Neighbourhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"


In [4]:
df.describe(include='all')

,Postal Code,Borough,Neighbourhood
count,103,103,103
unique,103,10,99
top,M2H,North York,Downsview
freq,1,24,4


All postal codes are unique--neighborhoods are already merged into a single postal code, so no need to alter the data frame.

In [5]:
df.shape

(103, 3)

## Part 2: Add neighborhood locations [](#item2) 

Initially tried using `geocoder`, but it was not working properly so I'm downloading the CSV of data instead. 

In [6]:
import wget
wget.download('http://cocl.us/Geospatial_data')

'Geospatial_Coordinates (4).csv'

In [7]:
geolocs = pd.read_csv('Geospatial_Coordinates.csv')
geolocs.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


Merge two data frames together by postal code using a smart merge (not just concatenating the columns)

In [8]:
df = df.merge(geolocs, on='Postal Code', how='right')
df

,Postal Code,Borough,Neighbourhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
...,...,...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North",43.653654,-79.506944
99,M4Y,Downtown Toronto,Church and Wellesley,43.665860,-79.383160
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C...",43.662744,-79.321558
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu...",43.636258,-79.498509


## Part 3: Explore and cluster neighborhoods [](#item3) 

Visualize the neighborhood locations

In [9]:
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
# import k-means from clustering stage
from sklearn.cluster import KMeans
import folium # map rendering library

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

In [10]:
address = 'Toronto, ON'

geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [11]:
# create map of Toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, label in zip(df['Latitude'], df['Longitude'], df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

### Now start to work with foursquare

(Hiding my credentials in the public GitHub repo)

In [12]:
CLIENT_ID = '#####' # your Foursquare ID
CLIENT_SECRET = '#####' # your Foursquare Secret
VERSION = '#####' # Foursquare API version
LIMIT = 100

Copy example function from lab to get venues near each neighborhood

In [13]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [14]:
toronto_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Parkwoods
Victoria Village
Regent Park, Harbourfront
Lawrence Manor, Lawrence Heights
Queen's Park, Ontario Provincial Government
Islington Avenue, Humber Valley Village
Malvern, Rouge
Don Mills
Parkview Hill, Woodbine Gardens
Garden District, Ryerson
Glencairn
West Deane Park, Princess Gardens, Martin Grove, Islington, Cloverdale
Rouge Hill, Port Union, Highland Creek
Don Mills
Woodbine Heights
St. James Town
Humewood-Cedarvale
Eringate, Bloordale Gardens, Old Burnhamthorpe, Markland Wood
Guildwood, Morningside, West Hill
The Beaches
Berczy Park
Caledonia-Fairbanks
Woburn
Leaside
Central Bay Street
Christie
Cedarbrae
Hillcrest Village
Bathurst Manor, Wilson Heights, Downsview North
Thorncliffe Park
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Scarborough Village
Fairview, Henry Farm, Oriole
Northwood Park, York University
East Toronto, Broadview North (Old East York)
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
Kennedy Park, Ionview, East Birchmo

In [15]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Parkwoods,43.753259,-79.329656,Brookbanks Park,43.751976,-79.332140,Park
1,Parkwoods,43.753259,-79.329656,649 Variety,43.754513,-79.331942,Convenience Store
2,Parkwoods,43.753259,-79.329656,Variety Store,43.751974,-79.333114,Food & Drink Shop
3,Victoria Village,43.725882,-79.315572,Victoria Village Arena,43.723481,-79.315635,Hockey Arena
4,Victoria Village,43.725882,-79.315572,Tim Hortons,43.725517,-79.313103,Coffee Shop


Let's do clustering entirely by restaurants! Will filter venue category to only include restaurants

In [16]:
toronto_res = toronto_venues[toronto_venues['Venue Category'].str.contains("Restaurant")]
toronto_res.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
5,Victoria Village,43.725882,-79.315572,Portugril,43.725819,-79.312785,Portuguese Restaurant
6,Victoria Village,43.725882,-79.315572,The Frig,43.727051,-79.317418,French Restaurant
13,"Regent Park, Harbourfront",43.654260,-79.360636,Impact Kitchen,43.656369,-79.356980,Restaurant
30,"Regent Park, Harbourfront",43.654260,-79.360636,Cluny Bistro & Boulangerie,43.650565,-79.357843,French Restaurant
33,"Regent Park, Harbourfront",43.654260,-79.360636,El Catrin,43.650601,-79.358920,Mexican Restaurant


In [17]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_res[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
toronto_onehot['Neighborhood'] = toronto_res['Neighborhood']

# move neighborhood column to the first column
fixed_columns = [toronto_onehot.columns[-1]] + list(toronto_onehot.columns[:-1])
toronto_onehot = toronto_onehot[fixed_columns]

# remove column for generic "Restaurant"
del toronto_onehot['Restaurant']

toronto_onehot.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,...,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
5,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
6,Victoria Village,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
13,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
30,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
33,"Regent Park, Harbourfront",0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group neighborhoods to get restaurant frequencies

In [18]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,Colombian Restaurant,...,New American Restaurant,Portuguese Restaurant,Ramen Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant
0,Agincourt,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
1,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.333333,0.0,0.000000,0.0,0.000000,0.0
2,Bayview Village,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.5,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0
3,"Bedford Park, Lawrence Manor East",0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.083333,0.0,0.083333,0.0,0.000000,0.0
4,Berczy Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.153846,0.076923,0.0,0.076923,0.0,0.076923,0.0


Now let's see if we can cluster neighborhoods based on their restaurant offerings.

In [19]:
# try another method using wcss  https://jtemporal.com/kmeans-and-elbow-method/
import math 

def calculate_wcss(data):
        wcss = []
        for n in range(2, 11): # limit to 11 to keep a manageable number of clusters
            kmeans = KMeans(n_clusters=n)
            kmeans.fit(X=data)
            wcss.append(kmeans.inertia_)
    
        return wcss

def optimal_number_of_clusters(wcss):
    x1, y1 = 2, wcss[0]
    x2, y2 = 20, wcss[len(wcss)-1]

    distances = []
    for i in range(len(wcss)):
        x0 = i+2
        y0 = wcss[i]
        numerator = abs((y2-y1)*x0 - (x2-x1)*y0 + x2*y1 - y2*x1)
        denominator = math.sqrt((y2 - y1)**2 + (x2 - x1)**2)
        distances.append(numerator/denominator)
    
    return distances.index(max(distances)) + 2

X = toronto_grouped.drop('Neighborhood', 1)
# calculating the within clusters sum-of-squares for 19 cluster amounts
sum_of_squares = calculate_wcss(X)
    
# calculating the optimal number of clusters
n = optimal_number_of_clusters(sum_of_squares)
n

10

In [20]:
# set number of clusters
kclusters = 10

toronto_clustering = toronto_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(toronto_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([7, 1, 2, 1, 1, 1, 3, 1, 1, 1], dtype=int32)

In [21]:
# add clustering labels
toronto_grouped.insert(0, 'Cluster Labels', kmeans.labels_)

toronto_merged = toronto_grouped
toronto_df = df[['Neighbourhood','Latitude','Longitude']].rename(columns={"Neighbourhood": "Neighborhood"})

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.merge(toronto_df, on='Neighborhood')

toronto_merged.head() # check the last columns!

,Cluster Labels,Neighborhood,Afghan Restaurant,American Restaurant,Asian Restaurant,Belgian Restaurant,Brazilian Restaurant,Cajun / Creole Restaurant,Caribbean Restaurant,Chinese Restaurant,...,Ramen Restaurant,Seafood Restaurant,Sushi Restaurant,Taiwanese Restaurant,Thai Restaurant,Theme Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Latitude,Longitude
0,7,Agincourt,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,43.794200,-79.262029
1,1,"Bathurst Manor, Wilson Heights, Downsview North",0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.333333,0.0,0.000000,0.0,0.000000,0.0,43.754328,-79.442259
2,2,Bayview Village,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.5,...,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,43.786947,-79.385975
3,1,"Bedford Park, Lawrence Manor East",0.0,0.083333,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.000000,0.083333,0.0,0.083333,0.0,0.000000,0.0,43.733283,-79.419750
4,1,Berczy Park,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.153846,0.076923,0.0,0.076923,0.0,0.076923,0.0,43.644771,-79.373306


Visualize clustering results

In [22]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=10)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighborhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

What are the clusters telling us?

In [24]:
toronto_long = toronto_merged.reset_index()
toronto_long = pd.melt(toronto_long, id_vars=['Cluster Labels','Neighborhood','Latitude','Longitude'])
# remove index rows
toronto_long = toronto_long[~toronto_long.variable.str.contains("index")]

In [25]:
freqs = toronto_long.groupby(['Cluster Labels','variable'])['value'].mean().reset_index()
freqs.sort_values(['value'],ascending=False,inplace=True)

In [26]:
for cluster in range(0,10):
    print(freqs[(freqs['Cluster Labels'] == cluster) & (freqs['value'] > 0)])

    Cluster Labels                       variable     value
24               0              Indian Restaurant  0.358333
7                0             Chinese Restaurant  0.162500
17               0           Fast Food Restaurant  0.150000
32               0      Middle Eastern Restaurant  0.133333
44               0  Vegetarian / Vegan Restaurant  0.083333
45               0          Vietnamese Restaurant  0.062500
    Cluster Labels                         variable     value
72               1               Italian Restaurant  0.107467
47               1              American Restaurant  0.071301
86               1                 Sushi Restaurant  0.065311
73               1              Japanese Restaurant  0.058280
77               1               Mexican Restaurant  0.054435
88               1                  Thai Restaurant  0.040865
63               1             Fast Food Restaurant  0.040021
65               1                French Restaurant  0.032137
85               1    

**Cluster 0 is characterized by a prevalence of Indian restaurants, as well as Chinese, fast food, and Middle Eastern restaurants. Cluster 1 represents areas that have a large number of different kinds of restaurants--these are likely high-population areas. Clusters 2-9 contain only one or two kinds of restaurants.**